In [1]:
import pandas as pd
import sklearn
from matplotlib import pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LassoCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import multilabel_confusion_matrix

%matplotlib inline

import statsmodels.api as sm
from statsmodels.api import OLS

In [2]:
crime_property_light_population= pd.read_csv("./model_data/crime_light_density.csv")
crime_property_light_population.head()

predictors = ['OFFENSE_CODE_GROUP','SHOOTING', 'MONTH', 'DAY_OF_WEEK', 'HOUR', 'Population density (per square mile of land area)', 'BLDG_VAL', 'LAND_VAL','light_density']
df = crime_property_light_population[predictors]
df = df.dropna()

In [19]:
#Let's label encode & one-hot encode the Categoricals (SHOOTING, DAY_OF_WEEK)

enc = OneHotEncoder(handle_unknown='ignore')

le = LabelEncoder() 
  
df['SHOOTING']= le.fit_transform(df['SHOOTING'])
df['DAY_OF_WEEK']= le.fit_transform(df['DAY_OF_WEEK']) 
df['OFFENSE_CODE_GROUP'] = le.fit_transform(df['OFFENSE_CODE_GROUP'])

In [20]:
df_cat = enc.fit_transform(df[["MONTH","SHOOTING","DAY_OF_WEEK"]]).toarray()

df_cat = pd.DataFrame(df_cat, columns=enc.get_feature_names(['MONTH','SHOOTING', 'DAY_OF_WEEK']))
df = pd.concat([df, df_cat], axis=1)
df = df.dropna()

df.head()

,OFFENSE_CODE_GROUP,SHOOTING,MONTH,DAY_OF_WEEK,HOUR,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density,SHOOTING_0,...,DAY_OF_WEEK_6.0,SHOOTING_0,SHOOTING_1,DAY_OF_WEEK_0,DAY_OF_WEEK_1,DAY_OF_WEEK_2,DAY_OF_WEEK_3,DAY_OF_WEEK_4,DAY_OF_WEEK_5,DAY_OF_WEEK_6
0,4.0,0.0,10.0,3.0,9.0,6206.9,455.644447,21.073507,0.437849,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,4.0,0.0,10.0,2.0,20.0,6206.9,455.644447,21.073507,0.437849,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,4.0,0.0,10.0,2.0,15.0,16212.0,2250.588503,63.554783,0.582893,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4.0,0.0,10.0,2.0,4.0,32724.1,14799.401260,480.463801,0.656585,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,4.0,0.0,10.0,6.0,8.0,12192.0,278.377510,165.249738,0.678941,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [21]:

X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'OFFENSE_CODE_GROUP'], 
                                                         df.OFFENSE_CODE_GROUP, test_size=0.2, 
                                                         random_state = 109, 
                                                         stratify = df.OFFENSE_CODE_GROUP)

In [22]:

X_train.head()

,SHOOTING,MONTH,DAY_OF_WEEK,HOUR,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density,SHOOTING_0,SHOOTING_1,...,DAY_OF_WEEK_6.0,SHOOTING_0,SHOOTING_1,DAY_OF_WEEK_0,DAY_OF_WEEK_1,DAY_OF_WEEK_2,DAY_OF_WEEK_3,DAY_OF_WEEK_4,DAY_OF_WEEK_5,DAY_OF_WEEK_6
6672,0.0,9.0,4.0,2.0,32724.1,14799.401260,480.463801,0.656585,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
170250,0.0,11.0,2.0,19.0,12277.3,3322.718539,24.693322,0.598427,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
116902,0.0,6.0,3.0,17.0,24074.6,5490.169358,260.905478,0.728900,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
45254,0.0,4.0,4.0,4.0,15871.3,4156.595899,36.813618,0.684760,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
347751,0.0,12.0,3.0,1.0,24074.6,5490.169358,260.905478,0.728900,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [23]:
#Let's scale the variables
min_max_scaler = MinMaxScaler()
numerical = ["HOUR", "BLDG_VAL", "LAND_VAL", "light_density", "MONTH", "Population density (per square mile of land area)" ]
X_normalized = (X_train - X_train.min(axis=0)) / (X_train.max(axis=0) - X_train.min(axis=0))
min_max_scaler.fit(X_train)
X_normalized_tst = pd.DataFrame(min_max_scaler.transform(X_test))
X_normalized_tst.columns = X_test.columns


display(X_normalized.head())
display(X_normalized_tst.head())


,SHOOTING,MONTH,DAY_OF_WEEK,HOUR,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density,SHOOTING_0,SHOOTING_1,...,DAY_OF_WEEK_6.0,SHOOTING_0,SHOOTING_1,DAY_OF_WEEK_0,DAY_OF_WEEK_1,DAY_OF_WEEK_2,DAY_OF_WEEK_3,DAY_OF_WEEK_4,DAY_OF_WEEK_5,DAY_OF_WEEK_6
6672,0.0,0.727273,0.666667,0.086957,0.458798,0.115336,0.606003,0.527372,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
170250,0.0,0.909091,0.333333,0.826087,0.150906,0.025304,0.004775,0.447331,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
116902,0.0,0.454545,0.500000,0.739130,0.328552,0.042308,0.316373,0.626896,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
45254,0.0,0.272727,0.666667,0.173913,0.205025,0.031846,0.020764,0.566148,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
347751,0.0,1.000000,0.500000,0.043478,0.328552,0.042308,0.316373,0.626896,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


,SHOOTING,MONTH,DAY_OF_WEEK,HOUR,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density,SHOOTING_0,SHOOTING_1,...,DAY_OF_WEEK_6.0,SHOOTING_0,SHOOTING_1,DAY_OF_WEEK_0,DAY_OF_WEEK_1,DAY_OF_WEEK_2,DAY_OF_WEEK_3,DAY_OF_WEEK_4,DAY_OF_WEEK_5,DAY_OF_WEEK_6
0,0.0,0.454545,0.333333,0.478261,0.106907,0.063183,0.830593,0.680005,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.818182,0.666667,0.565217,0.386443,0.063450,1.000000,0.725570,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.545455,0.833333,0.043478,0.150906,0.025304,0.004775,0.447331,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.818182,0.333333,0.869565,0.190164,0.007405,0.013439,0.505555,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.000000,0.500000,0.043478,0.395815,0.485981,0.364563,0.747450,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Let's first start with Logistic Model

logreg = LogisticRegression(random_state = 0, solver='lbfgs', multi_class='multinomial', max_iter = 1000)
logreg.fit(X_normalized, y_train)
y_pred = logreg.predict(X_normalized_tst)

In [ ]:
mcm = multilabel_confusion_matrix(y_test, y_pred, sample_weight=None, labels=None, samplewise=False)
logreg.score(X_normalized, y_test)

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 7).fit(X_normalized, y_train) 
  
# accuracy on X_test 
accuracy = knn.score(X_normalized_tst, y_test) 

In [ ]:
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test, y_pred)

print(cm)

In [ ]:
import pickle

pickle.dump(logreg, open('logreg', 'wb'))
pickle.dump(knn, open('knn', 'wb'))